# <center> <h1> [🧶 Связка `Lightning` + `ClearML` в CV. 🖼](https://stepik.org/lesson/1500763/step/1?unit=1520877)

### Оглавление ноутбука
<img src='https://i.pinimg.com/originals/8b/1f/3b/8b1f3bc02301ab06c15fe48b84de4c48.gif' align="right" width="508" height="428" >
<br>

<p><font size="3" face="Arial" font-size="large"><ul type="square">
    
<li><a href="#p2">🎨 Задача классификации изображений</a></li>
<li><a href="#p7">🎛 Файнтюнинг предобученной модели.</a></li>
<li><a href="#p3">🧶 Связка в задаче семантической сегментации 🧩</a></li>
<li><a href="#p6">🧸 Выводы и заключения ✅ </a></li>


    
</ul></font></p>

### <center> 🧑‍🎓Разберем как связка **PyTorch Lightning** c **ClearML** может работать в CV-задачах. 🪄

<div class="alert alert-info">

Связка **ClearML** с **PyTorch Lightning** так же отлично показывает себя в задачах из CV-домена.

* Удобно отслеживать прогресс обучения сетки, особенно, при обучении на удалённых севрверах, а не в ноутбуках.
* Легко запустить файнтюнинг предобученного бэкбона
* Даже можно отслеживать прогресс обучения генеративных моделей (GAN и.т.п)
* Плюс все преимущества, которые рассматривали в прошлом уроке.

###  <center> Установка библиотек и подгрузка ключей

In [8]:
!pip install clearml tensorboard lightning torchvision torchmetrics -q

In [9]:
import logging
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchmetrics import Accuracy
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

from lightning import LightningModule, Trainer, seed_everything
from lightning.pytorch.callbacks import LearningRateMonitor, ModelCheckpoint
from lightning.pytorch.loggers import TensorBoardLogger

In [ ]:
# from getpass import getpass
# Введите поочерёдно полученные ключи в появившемся окне (код изменять не нужно)
# access_key = getpass(prompt="Введите API Access токен: ")
# secret_key = getpass(prompt="Введите API Secret токен: ")

In [ ]:
# %%capture
# #  Не показывать свои api-ключи
# %env CLEARML_WEB_HOST=https://app.clear.ml/
# %env CLEARML_API_HOST=https://api.clear.ml
# %env CLEARML_FILES_HOST=https://files.clear.ml

# %env CLEARML_API_ACCESS_KEY=$access_key
# %env CLEARML_API_SECRET_KEY=$secret_key

In [10]:
%env CLEARML_WEB_HOST=https://app.clear.ml/
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=UR73VVAQC4EUA46EA09ZMIM08YT21P
%env CLEARML_API_SECRET_KEY=ZC2NeA8YNAxcEfepGgjZ9RTI0yJNx1E24R3W3dqYx4P5LWTRFOWuwCQhHB2pSwCxMvg

env: CLEARML_WEB_HOST=https://app.clear.ml/
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml
env: CLEARML_API_ACCESS_KEY=UR73VVAQC4EUA46EA09ZMIM08YT21P
env: CLEARML_API_SECRET_KEY=ZC2NeA8YNAxcEfepGgjZ9RTI0yJNx1E24R3W3dqYx4P5LWTRFOWuwCQhHB2pSwCxMvg


# <center> 🎨 Задача классификации изображений </center>

<div class="alert alert-info">

Подключаем наш родной СlearML и сразу загружаем в него конфиг

In [11]:
from clearml import Task

# Инициализация задачи в ClearML
task = Task.init(
    project_name="Image Classification",
    task_name="CIFAR10 Training with Image Logging",
    task_type=Task.TaskTypes.training,
)

ClearML Task: created new task id=5bb19febe47c4d6dbf3345044a85514c
2025-04-15 23:49:59,839 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/05a161c5d9a4441682554ceef036ed08/experiments/5bb19febe47c4d6dbf3345044a85514c/output/log


In [12]:
config = {
    "num_epochs": 3,
    "batch_size": 24,
    "learning_rate": 0.001,
    "dropout_rate": 0.25,
    "num_images_to_log": 5,  # Количество изображений для логирования
}
config = task.connect(config)  # Добавление гиперпараметров в ClearML
print("Hyperparameters:", config)

Hyperparameters: {'num_epochs': 3, 'batch_size': 24, 'learning_rate': 0.001, 'dropout_rate': 0.25, 'num_images_to_log': 5}


<div class="alert alert-info">

Напишем и обучим собственную сетку на датасете `CIFAR10`!

Используем датасет CIFAR10

In [13]:
# Классы CIFAR-10
classes = (
    "plane",
    "car",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
)

In [14]:
# Подготовка данных
transform = transforms.Compose([transforms.ToTensor()])

train_dataset = datasets.CIFAR10(
    root="./data", train=True, download=True, transform=transform
)
test_dataset = datasets.CIFAR10(
    root="./data", train=False, download=True, transform=transform
)

Files already downloaded and verified
Files already downloaded and verified


In [15]:
# Подготовка даталоадера
train_loader = DataLoader(
    train_dataset, batch_size=config["batch_size"], shuffle=True, num_workers=2
)
val_loader = DataLoader(
    test_dataset, batch_size=config["batch_size"], shuffle=False, num_workers=2
)

## <center> 💃 Модель (соберём простую архитектуру из 2-х Convolution слоёв)

In [16]:
class CIFAR10Model(LightningModule):
    def __init__(self, num_classes=10, dropout_rate=0.25, learning_rate=0.001):
        super().__init__()
        self.save_hyperparameters()
        self.model = nn.Sequential(
            nn.Conv2d(3, 6, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(6, 16, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Linear(16 * 6 * 6, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.Linear(84, num_classes),
        )
        self.loss_fn = nn.CrossEntropyLoss()
        self.accuracy = Accuracy(task="multiclass", num_classes=num_classes, top_k=1)

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self(inputs)
        loss = self.loss_fn(outputs, labels)
        self.log("train_loss", loss, on_step=True, on_epoch=True)
        task.get_logger().report_scalar("Loss", "train", loss.item(), self.global_step)
        return loss

    def validation_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self(inputs)
        loss = self.loss_fn(outputs, labels)
        acc = self.accuracy(outputs, labels)
        self.log("val_loss", loss, on_step=False, on_epoch=True)
        self.log("val_acc", acc, on_step=False, on_epoch=True)
        task.get_logger().report_scalar("Loss", "val", loss.item(), self.global_step)
        task.get_logger().report_scalar("Accuracy", "val", acc.item(), self.global_step)

        # Логирование изображений
        if batch_idx == 0:  # Логируем только для первого батча
            _, predicted = outputs.max(1)
            for i in range(min(len(inputs), config["num_images_to_log"])):
                img = transforms.ToPILImage()(inputs[i].cpu())
                pred_label = predicted[i].item()
                true_label = labels[i].item()
                title = f"True: {classes[true_label]}, Pred: {classes[pred_label]}"
                # Берём логгер из ClearML и его метод report_image()
                task.get_logger().report_image(
                    title=title,
                    series=f"Batch {self.current_epoch}",
                    iteration=self.global_step,
                    image=img,
                )

    def configure_optimizers(self):
        return optim.SGD(self.parameters(), lr=self.hparams.learning_rate, momentum=0.9)

### <center> 🚀 Запускаем обучение через тренера 💪

In [17]:
# Callbacks для мониторинга
callbacks = [
    ModelCheckpoint(monitor="val_acc", mode="max", save_top_k=1),
    LearningRateMonitor(logging_interval="step"),
]

In [18]:
seed_everything(42)

Seed set to 42


42

In [19]:
trainer = Trainer(
    max_epochs=config["num_epochs"],
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    devices="auto",
    callbacks=callbacks,
    log_every_n_steps=50,
)

Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [20]:
# Запуск обучения
model = CIFAR10Model(
    dropout_rate=config["dropout_rate"], learning_rate=config["learning_rate"]
)
trainer.fit(model, train_loader, val_loader)

You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name     | Type               | Params | Mode 
--------------------------------------------------------
0 | model    | Sequential         | 81.3 K | train
1 | loss_fn  | CrossEntropyLoss   | 0      | train
2 | accuracy | MulticlassAccuracy | 0      | train
--------------------------------------------------------
81.3 K    Trainable params
0         Non-trainable params
81.3 K    Total params
0.325     Total estimated model params size (MB)
16        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=3` reached.


In [21]:
# Не забываем закрыть таск
task.close()

In [22]:
task.get_output_log_web_page()

'https://app.clear.ml/projects/05a161c5d9a4441682554ceef036ed08/experiments/5bb19febe47c4d6dbf3345044a85514c/output/log'


<div class="alert alert-success">

* Если перейти на вкладку `DEBUG SAMPLES` экспермента увидим, что с простой архитектурой модель совсем плохо научилась определять классы изображений
  
<img src='../images/cifnet.png' align="center" height="428" >

# <center> 🎛 Файнтюнинг предобученной модели. </center>

<div class="alert alert-info">

  
* Далее попробуем взять предобученный бэкбон `ResNet18`
* Заодно посмотрим как можно реализовать `Transfer Learning` на `Lightning`!
* Сперва все так же делаем подготовку конфига **ClearML**

In [23]:
from clearml import Task

# ✅ Инициализация ClearML Task
task = Task.init(
    project_name="Image Classification",
    task_name="Fine-tuning ResNet18 on CIFAR10",
    task_type=Task.TaskTypes.training,
)

ClearML Task: created new task id=34790ff7ccbc492783af2998217e1d6d
ClearML results page: https://app.clear.ml/projects/05a161c5d9a4441682554ceef036ed08/experiments/34790ff7ccbc492783af2998217e1d6d/output/log
ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


In [24]:
# ✅ Гиперпараметры
config = {
    "num_epochs": 10,
    "batch_size": 32,
    "learning_rate": 1e-3,
    "fine_tune_start_epoch": 5,  # С какой эпохи размораживать бэкбон
    "num_classes": 10,
    "num_images_to_log": 5,  # Логирование картинок в ClearML
}
config = task.connect(config)
print("Hyperparameters:", config)

Hyperparameters: {'num_epochs': 10, 'batch_size': 32, 'learning_rate': 0.001, 'fine_tune_start_epoch': 5, 'num_classes': 10, 'num_images_to_log': 5}


## <center> 🤰 Модель (возьмём предобученный бэкбон ResNet18)

<div class="alert alert-info">

**🔹 Как работает Fine-Tuning?**
* В первые `fine_tune_start_epoch` эпохи тренируется только последний слой (fc).
* Затем размораживается весь `ResNet18` и обучается вся модель.
* 🔥 Это стандартный подход в `Transfer Learning`, который делает `fine-tuning` эффективнее и стабильнее! 🚀

In [27]:
# ✅ Lightning-модель для обучения ResNet18
class FineTuneResNet(LightningModule):
    def __init__(self, num_classes=10, learning_rate=1e-3, fine_tune_start_epoch=5):
        super().__init__()
        self.save_hyperparameters()

        # Загружаем предобученный ResNet18
        self.model = models.resnet18(pretrained=True)

        # Заменяем последний слой на 10 классов (CIFAR-10)
        in_features = self.model.fc.in_features
        self.model.fc = nn.Linear(in_features, num_classes)

        # Замораживаем веса бэкбона (кроме fc)
        for param in self.model.parameters():
            param.requires_grad = False

        for param in self.model.fc.parameters():
            param.requires_grad = True

        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self(inputs)
        loss = self.loss_fn(outputs, labels)
        self.log("train_loss", loss, on_step=True, on_epoch=True)

        # Логирование в ClearML
        task.get_logger().report_scalar("Loss", "train", loss.item(), self.global_step)

        return loss

    def validation_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self(inputs)
        loss = self.loss_fn(outputs, labels)
        acc = (outputs.argmax(dim=1) == labels).float().mean()

        self.log("val_loss", loss, on_epoch=True)
        self.log("val_acc", acc, on_epoch=True)

        task.get_logger().report_scalar("Loss", "val", loss.item(), self.global_step)
        task.get_logger().report_scalar("Accuracy", "val", acc.item(), self.global_step)

        # Логирование изображений в ClearML
        if batch_idx == 0:
            _, predicted = outputs.max(1)
            for i in range(min(len(inputs), config["num_images_to_log"])):
                img = transforms.ToPILImage()(inputs[i].cpu())
                pred_label = predicted[i].item()
                true_label = labels[i].item()
                title = f"True: {classes[true_label]}, Pred: {classes[pred_label]}"
                task.get_logger().report_image(
                    title=title,
                    series=f"Epoch {self.current_epoch}",
                    iteration=self.global_step,
                    image=img,
                )

    def configure_optimizers(self):
        return optim.Adam(self.model.fc.parameters(), lr=self.hparams.learning_rate)

    def on_epoch_start(self):
        # Размораживаем бэкбон после N-й эпохи
        if self.current_epoch >= self.hparams.fine_tune_start_epoch:
            for param in self.model.parameters():
                param.requires_grad = True
            print(f"🔓 Разморозка всех слоев модели на {self.current_epoch}-й эпохе")

<div class="alert alert-info">
    
**🔹 Что делает код?**
1. Загружает предобученную `ResNet18` из `torchvision.models`
2. Меняет последний слой на 10 классов (CIFAR-10)
3. Замораживает веса бэкбона (первые `fine_tune_start_epoch` эпох)
4. Размораживает всю модель после `fine_tune_start_epoch`
5. Логирует метрики и изображения в `ClearML`
6. Сохраняет лучшую модель по `val_acc`

<div class="alert alert-info">

**🔹 Что логируется в ClearML?**

- ✅ Метрики (`train_loss`, `val_loss`, `val_acc`)
- ✅ Отладочные изображения с предсказаниями
- ✅ Гиперпараметры эксперимента
- ✅ Модель с лучшей точностью

In [28]:
model = FineTuneResNet(
    num_classes=config["num_classes"],
    learning_rate=config["learning_rate"],
    fine_tune_start_epoch=config["fine_tune_start_epoch"],
)

model

/home/sinarian/venvs/sinara/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning:

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.

/home/sinarian/venvs/sinara/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning:

Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.



FineTuneResNet(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, trac

In [29]:
callbacks = [
    ModelCheckpoint(monitor="val_acc", mode="max", save_top_k=1),
    LearningRateMonitor(logging_interval="step"),
]

In [30]:
# ✅ Тренировочный процесс
trainer = Trainer(
    max_epochs=config["num_epochs"],
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    devices="auto",
    callbacks=callbacks,
    log_every_n_steps=50,
)

# ✅ Запуск обучения
trainer.fit(model, train_loader, val_loader)

Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name    | Type             | Params | Mode 
-----------------------------------------------------
0 | model   | ResNet           | 11.2 M | train
1 | loss_fn | CrossEntropyLoss | 0      | train
-----------------------------------------------------
5.1 K     Trainable params
11.2 M    Non-trainable params
11.2 M    Total params
44.727    Total estimated model params size (MB)
69        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [31]:
task.close()

In [32]:
task.get_output_log_web_page()

'https://app.clear.ml/projects/05a161c5d9a4441682554ceef036ed08/experiments/34790ff7ccbc492783af2998217e1d6d/output/log'

## <center> 👀 Посмотрим на валидационные изображения

    

<div class="alert alert-success">

* Видно как модель постепенно "умнеет" - перестаёт ошибаться в определении классов.
* Сначала ошибается во всех классах, потом правильно определяет `cat` и `ship`

<img src='../images/cif10.png'>

# <center> 🧶 Связка в задаче семантической сегментации 🧩</center>

<div class="alert alert-info">

Разберём пример решения задачи сегментации с использованием `PyTorch Lightning` и `ClearML`. Будем использовать модель `FCN-ResNet50` из `torchvision.models.segmentation` и обучать её на `VOC Segmentation Dataset (Pascal VOC 2012)`.

`VOC Segmentation Dataset (Pascal VOC 2012)` - содержит изображения с объектами 20 классов и 1 фоновый класс.

Так же из `torchmetrics` возьмём `JaccardIndex` для вычисления метрики `IoU`.

В итоге получится сильная связка: **⚡ Lightning + ClearML + torchmetrics**

In [33]:
# ✅ Инициализация задачи в ClearML
task = Task.init(
    project_name="VOC-Segmentation",
    task_name="Fine-Tuning FCN-ResNet50-VOC",
    task_type=Task.TaskTypes.training,
)

ClearML Task: created new task id=77ae83b4fb1d41f7bfc71221fc8c2830
ClearML results page: https://app.clear.ml/projects/3f6029130fe24501a93383579483c1df/experiments/77ae83b4fb1d41f7bfc71221fc8c2830/output/log
ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


In [34]:
# ✅ Гиперпараметры
config = {
    # Data parameters
    "dataset": "VOC2012",
    "data_root": "./data",
    "image_size": (520, 520),
    "batch_size": 8,
    "num_workers": 4,
    # Model parameters
    "model_name": "fcn_resnet50",
    "pretrained": True,
    "num_classes": 21,  # VOC has 20 classes + background
    # Training parameters
    "learning_rate": 0.001,
    "num_epochs": 10,
    "optimizer": "Adam",
    "scheduler": "ReduceLROnPlateau",
    "scheduler_params": {"mode": "min", "factor": 0.5, "patience": 2},
    # Transforms
    "image_normalization": {
        "mean": [0.485, 0.456, 0.406],
        "std": [0.229, 0.224, 0.225],
    },
    # Hardware
    "device": "gpu" if torch.cuda.is_available() else "cpu",
}
config = task.connect(config)
print("Hyperparameters:", config)

Hyperparameters: {'dataset': 'VOC2012', 'data_root': './data', 'image_size': (520, 520), 'batch_size': 8, 'num_workers': 4, 'model_name': 'fcn_resnet50', 'pretrained': True, 'num_classes': 21, 'learning_rate': 0.001, 'num_epochs': 10, 'optimizer': 'Adam', 'scheduler': 'ReduceLROnPlateau', 'scheduler_params': {'mode': 'min', 'factor': 0.5, 'patience': 2}, 'image_normalization': {'mean': [0.485, 0.456, 0.406], 'std': [0.229, 0.224, 0.225]}, 'device': 'gpu'}


In [35]:
# ✅ Функция загрузки данных (Pascal VOC 2012)
def get_dataloaders(batch_size):
    transform = transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize(
                mean=config["image_normalization"]["mean"],
                std=config["image_normalization"]["std"],
            ),
            transforms.Resize(config["image_size"], antialias=True),
        ]
    )

    target_transform = transforms.Compose(
        [
            transforms.PILToTensor(),
            transforms.Resize(
                config["image_size"], interpolation=transforms.InterpolationMode.NEAREST
            ),
        ]
    )

    train_dataset = datasets.VOCSegmentation(
        root=config["data_root"],
        year="2012",
        image_set="train",
        download=True,
        transform=transform,
        target_transform=target_transform,
    )
    val_dataset = datasets.VOCSegmentation(
        root=config["data_root"],
        year="2012",
        image_set="val",
        download=True,
        transform=transform,
        target_transform=target_transform,
    )

    # Log dataset information
    task.get_logger().report_text(
        f"Train dataset size: {len(train_dataset)} samples", level=logging.INFO
    )
    task.get_logger().report_text(
        f"Validation dataset size: {len(val_dataset)} samples", level=logging.INFO
    )

    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=config["num_workers"],
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=config["num_workers"],
    )

    return train_loader, val_loader

In [36]:
# Названия классов VOC
voc_class_names = [
    "background",
    "aeroplane",
    "bicycle",
    "bird",
    "boat",
    "bottle",
    "bus",
    "car",
    "cat",
    "chair",
    "cow",
    "diningtable",
    "dog",
    "horse",
    "motorbike",
    "person",
    "pottedplant",
    "sheep",
    "sofa",
    "train",
    "tvmonitor",
]
# Логируем названия классов
task.connect({"class_mapping": {i: name for i, name in enumerate(voc_class_names)}})

Unsupported key of type '<class 'int'>' found when connecting dictionary. It will be converted to str


{'class_mapping': {0: 'background',
  1: 'aeroplane',
  2: 'bicycle',
  3: 'bird',
  4: 'boat',
  5: 'bottle',
  6: 'bus',
  7: 'car',
  8: 'cat',
  9: 'chair',
  10: 'cow',
  11: 'diningtable',
  12: 'dog',
  13: 'horse',
  14: 'motorbike',
  15: 'person',
  16: 'pottedplant',
  17: 'sheep',
  18: 'sofa',
  19: 'train',
  20: 'tvmonitor'}}

In [37]:
# ✅ Залогируем трансформации
transforms_config = {
    "image_transforms": [
        "ToTensor()",
        f"Normalize(mean={config['image_normalization']['mean']}, std={config['image_normalization']['std']})",
        f"Resize(size={config['image_size']}, antialias=True)",
    ],
    "target_transforms": [
        "PILToTensor()",
        f"Resize(size={config['image_size']}, interpolation=InterpolationMode.NEAREST)",
    ],
}

# Log configuration to ClearML
task.connect_configuration({"transforms_config": transforms_config})
print("Transforms:", transforms_config)

Transforms: {'image_transforms': ['ToTensor()', 'Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])', 'Resize(size=(520, 520), antialias=True)'], 'target_transforms': ['PILToTensor()', 'Resize(size=(520, 520), interpolation=InterpolationMode.NEAREST)']}


In [38]:
# ✅ Модель сегментации с предобученным FCN-ResNet50
import torchmetrics
from PIL import Image
from torchvision.models.segmentation import FCN_ResNet50_Weights, fcn_resnet50


class SegmentationModel(LightningModule):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.num_classes = config["num_classes"]
        self.lr = config["learning_rate"]

        # Конфиг модели
        self.model_config = {
            "backbone": "resnet50",
            "head": "FCN",
            "pretrained": config["pretrained"],
            "num_classes": self.num_classes,
        }

        # Загружаем pretrained FCN-ResNet50
        if config["pretrained"]:
            self.model = fcn_resnet50(weights=FCN_ResNet50_Weights.DEFAULT)
        else:
            self.model = fcn_resnet50(weights=None)

        self.model.classifier[4] = nn.Conv2d(
            512, self.num_classes, kernel_size=(1, 1), stride=(1, 1)
        )

        # Метрики
        self.train_iou = torchmetrics.JaccardIndex(
            task="multiclass", num_classes=self.num_classes, ignore_index=255
        )
        self.val_iou = torchmetrics.JaccardIndex(
            task="multiclass", num_classes=self.num_classes, ignore_index=255
        )

        # Метрика IoU
        self.val_class_iou = torchmetrics.JaccardIndex(
            task="multiclass",
            num_classes=self.num_classes,
            average=None,
            ignore_index=255,
        )

        # ClearML logger
        self.clearml_logger = task.get_logger()

        # Логируем конфигурацию модели
        self.clearml_logger.report_text(
            f"Model Architecture: {self.model_config}", level=logging.INFO
        )

    def forward(self, x):
        return self.model(x)["out"]

    def setup(self, stage=None):
        self.validation_step_outputs = []

    def training_step(self, batch, batch_idx):
        images, masks = batch
        masks = masks.squeeze(1).long()  # Convert [B, 1, H, W] to [B, H, W]

        outputs = self(images)
        loss = F.cross_entropy(outputs, masks, ignore_index=255)

        # Вычисляем IoU
        preds = torch.argmax(outputs, dim=1)
        iou = self.train_iou(preds, masks)

        # Логируем метрики в ClearML
        self.clearml_logger.report_scalar(
            "Train/Loss", "CE", loss.item(), self.global_step
        )
        self.clearml_logger.report_scalar(
            "Train/IoU", "Mean", iou.item(), self.global_step
        )

        return loss

    def validation_step(self, batch, batch_idx):
        images, masks = batch
        masks = masks.squeeze(1).long()

        outputs = self(images)
        loss = F.cross_entropy(outputs, masks, ignore_index=255)

        # Вычмсляем IoU
        preds = torch.argmax(outputs, dim=1)
        iou = self.val_iou(preds, masks)

        # Calculate per-class IoU
        self.val_class_iou(preds, masks)

        # Логируем валидационные сэмплы
        if batch_idx == 0:
            self._log_images(images, masks, preds)

        result = {"val_loss": loss, "val_iou": iou}
        self.validation_step_outputs.append(result)
        return result

    def on_validation_epoch_end(self):
        outputs = self.validation_step_outputs
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        avg_iou = self.val_iou.compute()

        # Получаем IoU для каждого класса и логируем
        class_ious = self.val_class_iou.compute()
        for i, class_iou in enumerate(class_ious):
            self.clearml_logger.report_scalar(
                f"Val/ClassIoU",
                voc_class_names[i],
                class_iou.item(),
                self.current_epoch,
            )

        # Сбрасываем метрики для следующей эпохи
        self.val_iou.reset()
        self.val_class_iou.reset()

        self.clearml_logger.report_scalar(
            "Val/Loss", "CE", avg_loss.item(), self.current_epoch
        )
        self.clearml_logger.report_scalar(
            "Val/IoU", "Mean", avg_iou.item(), self.current_epoch
        )

        self.log("val_loss", avg_loss, prog_bar=True)
        self.log("val_iou", avg_iou, prog_bar=True)

    def _log_images(self, images, masks, predictions, max_imgs=4):
        """Log images with ClearML for visualization"""
        n_imgs = min(max_imgs, images.shape[0])

        # Создаём colormap для масок сегментации
        def colorize_mask(mask):
            cmap = plt.get_cmap("tab20", self.num_classes)
            colored = cmap(mask.cpu().numpy())
            return Image.fromarray((colored[:, :, :3] * 255).astype(np.uint8))

        for idx in range(n_imgs):
            # Оригинальное изображение
            img = images[idx].cpu().permute(1, 2, 0).numpy()
            img = np.clip(
                (
                    img * np.array(self.config["image_normalization"]["std"])
                    + np.array(self.config["image_normalization"]["mean"])
                ),
                0,
                1,
            )
            img = Image.fromarray((img * 255).astype(np.uint8))

            # Ground truth маска
            gt_mask = colorize_mask(masks[idx])

            # предсказанная маска
            pred_mask = colorize_mask(predictions[idx])

            self.clearml_logger.report_image(
                title="Validation Images",
                series="Input Image",
                iteration=self.global_step,
                image=img,
            )
            self.clearml_logger.report_image(
                title="Validation Images",
                series="GT Mask",
                iteration=self.global_step,
                image=gt_mask,
            )
            self.clearml_logger.report_image(
                title="Validation Images",
                series="Predicted Mask",
                iteration=self.global_step,
                image=pred_mask,
            )

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)

        if self.config["scheduler"] == "ReduceLROnPlateau":
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                optimizer,
                mode=self.config["scheduler_params"]["mode"],
                factor=self.config["scheduler_params"]["factor"],
                patience=self.config["scheduler_params"]["patience"],
                verbose=True,
            )
            return {
                "optimizer": optimizer,
                "lr_scheduler": {
                    "scheduler": scheduler,
                    "monitor": "val_loss",
                    "interval": "epoch",
                    "frequency": 1,
                },
            }
        else:
            return optimizer

<div class="alert alert-info">
    
**🔹 Как это работает?**
1. `FCN-ResNet50` из `torchvision`:

* Мы берем предобученный `FCN-ResNet50`.
* Так же можем начать обучение с нуля.
* Выходной слой переделываем для `num_classes` (21).

<div class="alert alert-info">
    
2. `JaccardIndex` из `torchmetrics`:

* Берём `JaccardIndex` для расчёта `IoU` - метрика в задаче сегментации
* Аккумулируем метрику на каждом шаге, потом считаем среднее занчение на всей эпохе.

<div class="alert alert-info">

3. Логирование в `ClearML`:

* гиперпараметры сетки
* `train_loss`, `val_loss`
* `train_iou`, `val_iou`
* Формат изображения: входное, маска Ground Truth, предсказанная маска.
* Вспомогательный метод `_log_images` позволяет визуально оценить, как хорошо модель сегментирует изображения, сравнивая предсказанные маски с истинными масками. Это полезно для диагностики и анализа производительности модели, а также для выявления возможных проблем в процессе обучения.
* Вспомогательный метод `on_validation_epoch_end` служит для агрегирования и логирования результатов валидации, что позволяет отслеживать производительность модели на протяжении обучения.

In [39]:
# ✅ Инициализация данных и модели
train_loader, val_loader = get_dataloaders(config["batch_size"])
model = SegmentationModel(config)

# ✅ Callbacks (сохранение лучшей модели, мониторинг LR)
callbacks = [
    ModelCheckpoint(
        monitor="val_iou",
        filename="fcn-resnet50-voc-{epoch:02d}-{val_iou:.3f}",
        save_top_k=2,
        mode="max",
        save_last=True,
    ),
    LearningRateMonitor(logging_interval="step"),
]

Using downloaded and verified file: ./data/VOCtrainval_11-May-2012.tar
Extracting ./data/VOCtrainval_11-May-2012.tar to ./data
Using downloaded and verified file: ./data/VOCtrainval_11-May-2012.tar
Extracting ./data/VOCtrainval_11-May-2012.tar to ./data
Train dataset size: 1464 samples
Validation dataset size: 1449 samples
Model Architecture: {'backbone': 'resnet50', 'head': 'FCN', 'pretrained': True, 'num_classes': 21}


In [40]:
# ✅ Тренировочный процесс
seed_everything(42)
trainer = Trainer(
    max_epochs=config["num_epochs"],
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    # devices="auto",
    callbacks=callbacks,
    log_every_n_steps=50,
)

Seed set to 42
Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [ ]:
# ✅ Запуск обучения
trainer.fit(model, train_loader, val_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name          | Type                   | Params | Mode 
-----------------------------------------------------------------
0 | model         | FCN                    | 35.3 M | train
1 | train_iou     | MulticlassJaccardIndex | 0      | train
2 | val_iou       | MulticlassJaccardIndex | 0      | train
3 | val_class_iou | MulticlassJaccardIndex | 0      | train
-----------------------------------------------------------------
35.3 M    Trainable params
0         Non-trainable params
35.3 M    Total params
141.289   Total estimated model params size (MB)
165       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [ ]:
# ✅ Не забываем закрыть таск
task.close()

In [ ]:
task.get_output_log_web_page()

### Посмотрим на валидационные сэмплы (вкладка `debug`)

<img src='../images/seg3.png'>
Видим, что модель постепенно обучается.

### Посмотрим на графики обучения (вкладка `scalars`)
<img src='../images/segscal.png'>

<div class="alert alert-success">

Таким образом, логирование в данном коде охватывает как количественные метрики (потери и IoU), так и качественные визуализации, что позволяет эффективно отслеживать процесс обучения и валидации модели сегментации.

## <center id="p6"> 🧸 Выводы и заключения ✅

<div class="alert alert-success">

В уроке рассмотрели способы как связка `Lightning + ClearML` поможет ускорить решение и отладку в CV-задачах:    
* Разобрали логирование задачи классификации изображений
* Файнтюнинг предобученного бэкбона
* Поняли как лучше организовать код для задачи сегментации
* В ДЗ попрактикуемся обучать GAN.

<div class="alert alert-info">
    
Связка `Lightning + ClearML` может быть очень полезной для любой задачи в CV-домене, особенно при обучении сеток на удалённой машине и в команде. <br>
Особенно сильно вы сможете прочувствовать всю мощь связки, если выполните практику по обучению `GAN`.